# Imperceptible attack on tabular data using LowProFool algorithm
In this notebook, we will learn how to execute imperceptible attack on tabular data with the LowProFool algorithm (https://arxiv.org/abs/1911.03274). We will use **iris flowers** and **breast cancer** datasets.

# Imports

In [ ]:
from art.estimators.classification.scikitlearn import ScikitlearnLogisticRegression
from art.estimators.classification.pytorch import PyTorchClassifier
from art.attacks.evasion import LowProFool

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable

___
# Data preparation

Firstly, we load the datasets, standardize them, and split into training and validation sets. We also choose the clipping values for both datasets.

In [ ]:
def standardize(data):
    """
    Get both the standardized data and the used scaler.
    """
    columns = data.columns
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(data)
    
    return pd.DataFrame(data=x_scaled, columns=columns), scaler

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

def get_train_and_valid(design_matrix, labels):
    """
    Split dataset into training and validation sets.
    """
    for train_idx, valid_idx in split.split(design_matrix, labels):
        X_train = design_matrix.iloc[train_idx].copy()
        X_valid = design_matrix.iloc[valid_idx].copy()
        y_train = labels.iloc[train_idx].copy()
        y_valid = labels.iloc[valid_idx].copy()

    return X_train, y_train, X_valid, y_valid

#### Loading and preparation of the iris flowers dataset

In [ ]:
iris = datasets.load_iris()
design_matrix_iris = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
labels_iris = pd.Series(data=iris['target'])
display(design_matrix_iris)

design_matrix_iris_scaled, iris_scaler = standardize(design_matrix_iris)

X_train_iris, y_train_iris, X_valid_iris, y_valid_iris =\
    get_train_and_valid(design_matrix_iris_scaled, labels_iris)

#### Loading and preparation of the breast cancer dataset

In [ ]:
cancer = datasets.load_breast_cancer()
design_matrix_cancer = pd.DataFrame(data=cancer['data'], columns=cancer['feature_names'])
labels_cancer = pd.Series(data=cancer['target'])
display(design_matrix_cancer)

design_matrix_cancer_scaled, cancer_scaler = standardize(design_matrix_cancer)

X_train_cancer, y_train_cancer, X_valid_cancer, y_valid_cancer =\
    get_train_and_valid(design_matrix_cancer_scaled, labels_cancer)

#### Clip-values

**Iris flowers dataset** - minimum and maximum values in training set.

In [ ]:
scaled_clip_values_iris = (
    np.array(X_train_iris.min()),
    np.array(X_train_iris.max())
)
print("Clip-values:")
print("  Lower bound:", scaled_clip_values_iris[0])
print("  Upper bound:", scaled_clip_values_iris[1])

print("\nClip-values in original scale:")
clip_values_iris = iris_scaler.inverse_transform(scaled_clip_values_iris)
print("  Lower bound:", clip_values_iris[0])
print("  Upper bound:", clip_values_iris[1])

**Breast cancer dataset** - 1 standard deviation boundary.

Note: Here, we create clip values such that all values should fall within the one standard deviation interval. Thanks to the dataset being priorly standardized, it is a trivial problem. Moreover clip values can be concisely expressed as just a single tuple `(-1., 1.)`.

In [ ]:
scaled_clip_values_cancer = (-1., 1.)

___
# Quick LowProFool example

A kickoff example using LowProFool to generate adversaries for SVC.

In [ ]:
# Import SVC
from sklearn.svm import SVC

# import ART wrapper for scikit-learn SVC
from art.estimators.classification.scikitlearn import ScikitlearnSVC

In [ ]:
# Initialize and fit SVC
svc_clf = SVC()
svc_clf.fit(X_train_cancer.values, y_train_cancer)

In [ ]:
# Wrap SVC using the wrapper
classif_svc = ScikitlearnSVC(
    model = svc_clf,
    clip_values = scaled_clip_values_cancer
)

In [ ]:
# Initialize LowProFool instance and fit feature importance
lpf_svc = LowProFool(
    classifier = classif_svc,
    n_steps    = 15,
    eta        = 15,
    lambd      = 1.75,
    eta_decay  = 0.985,
    verbose    = True
)
lpf_svc.fit_importances(X_train_cancer, y_train_cancer)

In [ ]:
# Create random array of samples to be used as adveraries
n_classes = lpf_svc.n_classes
targets = np.eye(n_classes)[np.array(
    y_valid_cancer.apply(lambda x: np.random.choice([i for i in range(n_classes) if i != x]))
)]

# Generate adversaries
adversaries = lpf_svc.generate(x=X_valid_cancer, y=targets)

# Check the success rate
expected = np.argmax(targets, axis=1)
predicted = np.argmax(classif_svc.predict(adversaries), axis=1)

correct = (expected == predicted)
success_rate = np.sum(correct) / correct.shape[0]
print("Test set size: {}".format(targets.shape[0]))
print("Success rate:  {:.2f}%".format(100*success_rate))

As you can see, one can easily generate good quality adversary examples in just a few lines of code.

___
# Extended LowProFool example

In this section we present you few more examples of `LowProFool` adversarial attacks carried out in a similar fashion, but employing different underlying models and on different datasets.

## Preparation of classifiers

### Logistic Regression

Training on iris flowers dataset

In [ ]:
log_regression_clf_iris = LogisticRegression()
log_regression_clf_iris.fit(X_train_iris.values, y_train_iris)

Training on breast cancer dataset

In [ ]:
log_regression_clf_cancer = LogisticRegression()
log_regression_clf_cancer.fit(X_train_cancer.values, y_train_cancer)

### Neural Network

In [ ]:
def get_nn_model(input_dimensions, hidden_neurons, output_dimensions):
    """
    Prepare PyTorch neural network.
    """
    return torch.nn.Sequential(
        nn.Linear(input_dimensions, hidden_neurons),
        nn.ReLU(),
        nn.Linear(hidden_neurons, output_dimensions),
        nn.Softmax(dim=1)
    )

loss_fn = torch.nn.MSELoss(reduction='sum')

def train_nn(nn_model, X, y, learning_rate, epochs):
    """
    Train provided neural network.
    """
    optimizer = optim.SGD(nn_model.parameters(), lr=learning_rate)
    
    for _ in range(epochs):
        y_pred = nn_model.forward(X)
        loss = loss_fn(y_pred, y)
        nn_model.zero_grad()
        loss.backward()
        
        optimizer.step()

Training on iris flowers dataset

In [ ]:
X = Variable(torch.FloatTensor(np.array(X_train_iris)))
y = Variable(torch.FloatTensor(np.eye(3)[y_train_iris]))
nn_model_iris = get_nn_model(4, 10, 3)
train_nn(nn_model_iris, X, y, 1e-4, 1000)

Training on breast cancer dataset

In [ ]:
X = Variable(torch.FloatTensor(np.array(X_train_cancer.values)))
y = Variable(torch.FloatTensor(np.eye(2)[y_train_cancer]))
nn_model_cancer = get_nn_model(30, 50, 2)
train_nn(nn_model_cancer, X, y, 1e-4, 1000)

## Actual usage of LowProFool

### Logistic Regression

In [ ]:
def lowprofool_generate_adversaries_test_lr(lowprofool, classifier, x_valid, y_valid):
    """
    Testing utility.
    """
    n_classes = lowprofool.n_classes
    
    # Generate targets
    target = np.eye(n_classes)[np.array(
        y_valid.apply(
            lambda x: np.random.choice([i for i in range(n_classes) if i != x]))
    )]
    
    # Generate adversaries
    adversaries = lowprofool.generate(x=x_valid, y=target)

    # Test - check the success rate
    expected = np.argmax(target, axis=1)
    predicted = np.argmax(classifier.predict_proba(adversaries), axis=1)
    correct = (expected == predicted)
    
    success_rate = np.sum(correct) / correct.shape[0]
    print("Success rate: {:.2f}%".format(100*success_rate))
    
    return adversaries

#### Iris flowers dataset test

In [ ]:
# Wrapping classifier into appropriate ART-friendly wrapper
logistic_regression_iris_wrapper = ScikitlearnLogisticRegression(
    model       = log_regression_clf_iris, 
    clip_values = scaled_clip_values_iris
)

# Creating LowProFool instance
lpf_logistic_regression_iris = LowProFool(
    classifier = logistic_regression_iris_wrapper, 
    eta        = 5,
    lambd      = 0.2, 
    eta_decay  = 0.9
)

# Fitting feature importance
lpf_logistic_regression_iris.fit_importances(X_train_iris, y_train_iris)

# Testing
results_lr_ir = lowprofool_generate_adversaries_test_lr(
    lowprofool = lpf_logistic_regression_iris,
    classifier = log_regression_clf_iris, 
    x_valid    = X_valid_iris, 
    y_valid    = y_valid_iris
)

Successful adversarial attack. Below we can see the original features and their classes, as well as the adversaries generated by `LowProFool` and predicted class-wise probabilities of them.

In [ ]:
def print_predictions(values, preds, max_features=4):
    """
    Utility function for printing predictions.
    """
    predictions = zip(list(map(lambda e: e[:max_features], values.tolist())), preds.tolist())
    
    for features, pred in predictions:
        print("Features[:{}]:".format(max_features))
        for i, val in enumerate(features):
            if i % 6 != 5: print("{:>10.4f}".format(val), end='')
            else:          print("{:>10.4f}\n".format(val), end='')
        if len(features) % 6 != 0: print()
        
        print("Prediction (probability -> class):")
        for val in pred:
            print("{:>8.3f}".format(val), end='')
        print("  ->  {}\n".format(np.argmax(pred)))

In [ ]:
print("=== Original values ===\n")

print_predictions(iris_scaler.inverse_transform(X_valid_iris[-3:].values), 
    log_regression_clf_iris.predict_proba(X_valid_iris[-3:]))
    
print("\n=== Adversaries (LowProFool results) ===\n")
    
print_predictions(iris_scaler.inverse_transform(results_lr_ir[-3:]), 
    log_regression_clf_iris.predict_proba(results_lr_ir[-3:]))

#### Breast cancer dataset test

In [ ]:
# Wrapping classifier into appropriate ART-friendly wrapper
logistic_regression_cancer_wrapper = ScikitlearnLogisticRegression(
    model       = log_regression_clf_cancer, 
    clip_values = scaled_clip_values_cancer
)

# Creating LowProFool instance
lpf_logistic_regression_cancer = LowProFool(
    classifier = logistic_regression_cancer_wrapper, 
    eta        = 5,
    lambd      = 0.2, 
    eta_decay  = 0.9
)

# Fitting feature importance
lpf_logistic_regression_cancer.fit_importances(X_train_cancer.values, y_train_cancer)

# Testing
results_lr_bc = lowprofool_generate_adversaries_test_lr(
    lowprofool = lpf_logistic_regression_cancer,
    classifier = log_regression_clf_cancer, 
    x_valid    = X_valid_cancer, 
    y_valid    = y_valid_cancer
)

In [ ]:
print("=== Original values ===\n")

print_predictions(cancer_scaler.inverse_transform(X_valid_cancer[-2:]), 
                  log_regression_clf_cancer.predict_proba(X_valid_cancer[-2:]), max_features=30)

print("\n=== Adversaries (LowProFool results) ===\n")

print_predictions(cancer_scaler.inverse_transform(results_lr_bc[-2:]), 
                  log_regression_clf_cancer.predict_proba(results_lr_bc[-2:]), max_features=30)

### Neural Network

In [ ]:
def lowprofool_generate_adversaries_test_nn(lowprofool, classifier, x_valid, y_valid):
    """
    Testing utility.
    """
    n_classes = lowprofool.n_classes
    
    # Generate targets
    target = np.eye(n_classes)[np.array(
        y_valid.apply(
            lambda x: np.random.choice([i for i in range(n_classes) if i != x]))
    )]
    
    # Generate adversaries
    adversaries = lowprofool.generate(x=x_valid, y=target)

    # Test - check the success rate
    expected = np.argmax(target, axis=1)
    x = Variable(torch.from_numpy(adversaries.astype(np.float32)))
    predicted = np.argmax(classifier.forward(x).detach().numpy(), axis=1)
    correct = (expected == predicted)
    
    success_rate = np.sum(correct) / correct.shape[0]
    print("Success rate: {:.2f}%".format(100*success_rate))
    
    return adversaries

#### Iris flowers dataset test

In [ ]:
# Wrapping classifier into appropriate ART-friendly wrapper
# (in this case it is PyTorch NN classifier wrapper from ART)
neural_network_iris_wrapper = PyTorchClassifier(
    model       = nn_model_iris, 
    loss        = loss_fn,
    input_shape = (4,),
    nb_classes  = 3,
    clip_values = scaled_clip_values_iris
)

# Creating LowProFool instance
lpf_neural_network_iris = LowProFool(
    classifier = neural_network_iris_wrapper,
    n_steps    = 100,
    eta        = 7,
    lambd      = 1.75, 
    eta_decay  = 0.95
)

# Fitting feature importance
lpf_neural_network_iris.fit_importances(X_train_iris, y_train_iris)

# Testing
results_nn_ir = lowprofool_generate_adversaries_test_nn(
    lowprofool = lpf_neural_network_iris,
    classifier = nn_model_iris, 
    x_valid    = X_valid_iris, 
    y_valid    = y_valid_iris
)

In [ ]:
print("=== Original values ===\n")

print_predictions(iris_scaler.inverse_transform(X_valid_iris[:3].values),
      neural_network_iris_wrapper.predict(X_valid_iris[:3].values.astype(np.float32)))

print("\n=== Adversaries (LowProFool results) ===\n")

print_predictions(iris_scaler.inverse_transform(results_nn_ir[:3]), 
      neural_network_iris_wrapper.predict(results_nn_ir.astype(np.float32)[:3]))

#### Breast cancer dataset test

In [ ]:
# Wrapping classifier into appropriate ART-friendly wrapper
# (in this case it is PyTorch NN classifier wrapper from ART)
neural_network_cancer_wrapper = PyTorchClassifier(
    model       = nn_model_cancer, 
    loss        = loss_fn, 
    input_shape = (30,),
    nb_classes  = 2,
    clip_values = scaled_clip_values_cancer
)

# Creating LowProFool instance
lpf_neural_network_cancer = LowProFool(
    classifier = neural_network_cancer_wrapper,
    n_steps    = 200,
    eta        = 10,
    lambd      = 2, 
    eta_decay  = 0.99
)

# Fitting feature importance
lpf_neural_network_cancer.fit_importances(X_train_cancer, y_train_cancer)

# Testing
results_nn_bc = lowprofool_generate_adversaries_test_nn(
    lowprofool = lpf_neural_network_cancer,
    classifier = nn_model_cancer, 
    x_valid    = X_valid_cancer, 
    y_valid    = y_valid_cancer
)

In [ ]:
print("=== Original values ===\n")

print_predictions(
    cancer_scaler.inverse_transform(X_valid_cancer[-2:]),
    neural_network_cancer_wrapper.predict(X_valid_cancer[-2:].values.astype(np.float32)),
    max_features=30
)

print("\n=== Adversaries (LowProFool results) ===\n")

print_predictions(
    cancer_scaler.inverse_transform(results_nn_bc[-2:]), 
    neural_network_cancer_wrapper.predict(results_nn_bc.astype(np.float32)[-2:]),
    max_features=30
)